In [129]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from scipy.stats import lognorm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import SequentialFeatureSelector
%matplotlib inline
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [71]:
df = pd.read_csv('train-2.csv', converters={'features': eval})
df = df.rename(str.lower, axis='columns',)
df = df.set_index('id')
display(df)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target
id,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25582,1.0,1,14fdc4b01ae44b025f6c4d28c9097e5f,2016-06-16 02:12:57,Newly renovated bedroom apartment located off ...,29th St,[No pets],40.7653,7166867,-73.9248,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7166867_625fde3...,1950,30-95 29th St,low
50013,1.0,0,9b6cf886379a2511f8c633c84028efe7,2016-05-10 03:17:32,All apartments are newly renovated featuring: ...,E 1st St,"[Pre-War, Laundry in Unit, Dishwasher, Hardwoo...",40.7239,6989251,-73.9901,5599e962719af3ccc2976855c2d5893c,['https://photos.renthop.com/2/6989251_42a4884...,2600,39 E 1st St,medium
111475,2.0,2,0,2016-04-21 03:29:35,"<![CDATA[2 bedrooms, 5110, Astoria / Long Isla...",50th Avenue,[],40.7437,6905568,-73.9585,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/6905568_01602e3...,5110,2-01 50th Avenue,low


In [72]:
y = df['target']
display(y)
X = df.drop(['target'], axis=1)
display(X)


id
57094     medium
33389     medium
60458        low
53048        low
592          low
           ...  
25582        low
50013     medium
111475       low
71184     medium
117473    medium
Name: target, Length: 34546, dtype: object

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
id,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25582,1.0,1,14fdc4b01ae44b025f6c4d28c9097e5f,2016-06-16 02:12:57,Newly renovated bedroom apartment located off ...,29th St,[No pets],40.7653,7166867,-73.9248,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7166867_625fde3...,1950,30-95 29th St
50013,1.0,0,9b6cf886379a2511f8c633c84028efe7,2016-05-10 03:17:32,All apartments are newly renovated featuring: ...,E 1st St,"[Pre-War, Laundry in Unit, Dishwasher, Hardwoo...",40.7239,6989251,-73.9901,5599e962719af3ccc2976855c2d5893c,['https://photos.renthop.com/2/6989251_42a4884...,2600,39 E 1st St
111475,2.0,2,0,2016-04-21 03:29:35,"<![CDATA[2 bedrooms, 5110, Astoria / Long Isla...",50th Avenue,[],40.7437,6905568,-73.9585,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/6905568_01602e3...,5110,2-01 50th Avenue


In [73]:
display(df.shape)
display(df.info())
display(df.isna().sum())

(34546, 15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34546 entries, 57094 to 117473
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bathrooms        34546 non-null  float64
 1   bedrooms         34546 non-null  int64  
 2   building_id      34546 non-null  object 
 3   created          34546 non-null  object 
 4   description      33509 non-null  object 
 5   display_address  34458 non-null  object 
 6   features         34546 non-null  object 
 7   latitude         34546 non-null  float64
 8   listing_id       34546 non-null  int64  
 9   longitude        34546 non-null  float64
 10  manager_id       34546 non-null  object 
 11  photos           34546 non-null  object 
 12  price            34546 non-null  int64  
 13  street_address   34542 non-null  object 
 14  target           34546 non-null  object 
dtypes: float64(3), int64(3), object(9)
memory usage: 4.2+ MB


None

bathrooms             0
bedrooms              0
building_id           0
created               0
description        1037
display_address      88
features              0
latitude              0
listing_id            0
longitude             0
manager_id            0
photos                0
price                 0
street_address        4
target                0
dtype: int64

In [74]:
display(df[df['target'] == 'low'])

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target
id,,,,,,,,,,,,,,,
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low
12930,1.0,2,0,2016-06-14 01:18:27,What is Lorem Ipsum?Lorem Ipsum is simply dumm...,East 11th Street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7278,7156298,-73.9809,70c9035cc23844487e1754b0d58577f5,[],3490,520 East 11th Street,low
103527,1.0,1,3269c2bdbbca7846582de8a174462ced,2016-04-09 04:39:21,Experience living in one of the best neighborh...,East 94th Street,"[Pre-War, Laundry in Building, Hardwood Floors]",40.7821,6849297,-73.9464,d2bce61e0e0079ebdc8c281e415e045b,['https://photos.renthop.com/2/6849297_789d89f...,1900,334 East 94th Street,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66346,1.0,1,96274288c84ddd7d5c5d8e425ee75027,2016-05-09 02:37:17,"Bicycle Room, Common Outdoor Space, Concierge,...",37th St.,"[Roof Deck, Dining Room, Balcony, Doorman, Ele...",40.7568,6985126,-73.9982,fa941827e1bb92613bece9a0a4494eee,['https://photos.renthop.com/2/6985126_c58dc10...,3900,505 37th St.,low
19753,1.0,3,1e6d49d4f0a9cf6b8985cc3f80bfc555,2016-06-12 12:32:12,Spacious NO BROKER FEE 3 Bed in Prewar...,West 162nd St,"[Elevator, No Fee]",40.8369,7151135,-73.9423,1ac657b9ded17ede9421ff43501f6e71,[],3395,566 West 162nd St,low
77858,1.0,1,d395becfad736011bb53155300eb238c,2016-05-28 05:34:00,"Location, Location, Location!!! If the perfect...",John Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7083,7080474,-74.0061,2aa9bfa5f67ed9997ea341dee8a3a271,['https://photos.renthop.com/2/7080474_8a885a4...,2600,99 John Street,low


In [75]:
high_percent = len(df[df['target'] == 'high']) * 100 / len(df)
medium_percent = len(df[df['target'] == 'medium']) * 100 / len(df)
low_percent = len(df[df['target'] == 'low']) * 100 / len(df)
display(f"high: {high_percent}%")
display(f"medium: {medium_percent}%")
display(f"low: {low_percent}%")


'high: 7.778035083656574%'

'medium: 22.752272332542116%'

'low: 69.4696925838013%'

In [76]:
df['temp_target'] = df['target'].map({'low' : low_percent, 'medium' : medium_percent + low_percent, 'high' : 1})
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target,temp_target
id,,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium,92.221965
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium,92.221965
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low,69.469693
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low,69.469693
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low,69.469693


In [77]:
df['most_frequent_house_comment'] = df.groupby('building_id')['temp_target'].transform(lambda x: x.sum() / len(x))
df.head()
# Значит многие одинаковые здания имеют одинаковые отзывы

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target,temp_target,most_frequent_house_comment
id,,,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium,92.221965,69.297433
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium,92.221965,80.845829
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low,69.469693,66.188957
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low,69.469693,67.339708
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low,69.469693,70.878405


In [78]:
df['most_frequent_rieltor_comment'] = df.groupby('manager_id')['temp_target'].transform(lambda x: x.sum() / len(x))
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target,temp_target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium,92.221965,69.297433,70.320227
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium,92.221965,80.845829,74.579835
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low,69.469693,66.188957,70.474858
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low,69.469693,67.339708,69.829174
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low,69.469693,70.878405,69.463040


In [79]:
df['description'] = df['description'].str.lower()
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target,temp_target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,a fabulous 3br in midtown west! perfect apar...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium,92.221965,69.297433,70.320227
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,renovated kitchen and bathroom!,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium,92.221965,80.845829,74.579835
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,rare and best deal on the market!!!! perfect s...,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low,69.469693,66.188957,70.474858
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,newly renovated flex 2 apartment offers the ne...,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low,69.469693,67.339708,69.829174
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,low fee apartments do not come around like thi...,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low,69.469693,70.878405,69.463040


In [80]:
def remove_trash(cur_str):
    cur_str = str(cur_str)
    nw_str = ""
    for elem in cur_str:
        if (elem >= 'a' and elem <= 'z') or elem == ' ':
            nw_str += elem

    return nw_str.split(' ')

df['description'] = df['description'].apply(remove_trash)
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target,temp_target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,"[a, fabulous, br, in, midtown, west, , , perfe...",HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium,92.221965,69.297433,70.320227
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,"[renovated, kitchen, and, bathroom]",55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium,92.221965,80.845829,74.579835
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,"[rare, and, best, deal, on, the, market, perfe...",W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low,69.469693,66.188957,70.474858
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,"[newly, renovated, flex, , apartment, offers, ...",John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low,69.469693,67.339708,69.829174
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,"[low, fee, apartments, do, not, come, around, ...",West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low,69.469693,70.878405,69.463040


In [81]:
description_words_weight = dict()
description_words_count = dict()
for index, row in df.iterrows():
    for word in row.description:
        if len(word) < 3:
            continue
        if word in description_words_weight:
            description_words_weight[word] += row.temp_target
            description_words_count[word] += 1
        else:
            description_words_weight[word] = row.temp_target
            description_words_count[word] = 1
        
for key, value in description_words_weight.items():
    description_words_weight[key] = value / (description_words_count[key] * 100)

display(description_words_weight['good'])

0.6830262066676251

In [82]:
def words_weight(x):
    res = 0.0
    for word in x:
        if word in description_words_weight:
            res += description_words_weight[word]
    if len(x) > 0:
        res /= len(x)
    return res

df['description'] = df['description'].apply(words_weight)
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target,temp_target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,0.548578,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium,92.221965,69.297433,70.320227
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,0.693230,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium,92.221965,80.845829,74.579835
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,0.542103,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low,69.469693,66.188957,70.474858
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,0.510558,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low,69.469693,67.339708,69.829174
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,0.533959,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low,69.469693,70.878405,69.463040


In [83]:
str.lower("KeK")

'kek'

In [84]:
features_words_weight = dict()
features_words_count = dict()
for index, row in df.iterrows():
    for word in row.features:
        word = str.lower(word)
        if word in features_words_weight:
            features_words_weight[word] += row.temp_target
            features_words_count[word] += 1
        else:
            features_words_weight[word] = row.temp_target
            features_words_count[word] = 1
        
        
for key, value in features_words_weight.items():
    features_words_weight[key] = value / (features_words_count[key] * 100)

display(features_words_weight['no fee'])
df.head()

0.6943055699304582

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target,temp_target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,0.548578,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium,92.221965,69.297433,70.320227
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,0.693230,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium,92.221965,80.845829,74.579835
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,0.542103,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low,69.469693,66.188957,70.474858
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,0.510558,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low,69.469693,67.339708,69.829174
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,0.533959,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low,69.469693,70.878405,69.463040


In [85]:
def features_weight(x):
    res = 0.0
    for word in x:
        word = str.lower(word)
        if word in features_words_weight:
            res += features_words_weight[word]
    if len(x) > 0:
        res /= len(x)
    return res

#display(features_weight(df.iloc[0, 6]))
df['features'] = df['features'].apply(features_weight)
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target,temp_target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,0.548578,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,0.700512,40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium,92.221965,69.297433,70.320227
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,0.693230,55 River Drive South,0.695180,40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium,92.221965,80.845829,74.579835
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,0.542103,W 77 Street,0.701212,40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low,69.469693,66.188957,70.474858
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,0.510558,John Street,0.704414,40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low,69.469693,67.339708,69.829174
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,0.533959,West 16th Street,0.703955,40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low,69.469693,70.878405,69.463040


In [86]:
df.drop(['building_id', 'created', 'display_address', 'latitude', 'listing_id', 'longitude', 'manager_id', 'photos', 'street_address'], inplace = True, axis = 1)
df.head()

,bathrooms,bedrooms,description,features,price,target,temp_target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,,
57094,1.0,3,0.548578,0.700512,4495,medium,92.221965,69.297433,70.320227
33389,1.0,1,0.693230,0.695180,2570,medium,92.221965,80.845829,74.579835
60458,1.0,0,0.542103,0.701212,1795,low,69.469693,66.188957,70.474858
53048,1.0,2,0.510558,0.704414,3400,low,69.469693,67.339708,69.829174
592,1.0,3,0.533959,0.703955,5695,low,69.469693,70.878405,69.463040


In [89]:
df.drop(['temp_target'], inplace = True, axis = 1)
df.head()

,bathrooms,bedrooms,description,features,price,target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,
57094,1.0,3,0.548578,0.700512,4495,medium,69.297433,70.320227
33389,1.0,1,0.693230,0.695180,2570,medium,80.845829,74.579835
60458,1.0,0,0.542103,0.701212,1795,low,66.188957,70.474858
53048,1.0,2,0.510558,0.704414,3400,low,67.339708,69.829174
592,1.0,3,0.533959,0.703955,5695,low,70.878405,69.463040


In [122]:
y = df['target'].map({'low' : 1, 'medium' : 2, 'high' : 3})
X = df.drop(['target'], axis = 1)
display(X.head())
display(y)

,bathrooms,bedrooms,description,features,price,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,
57094,1.0,3,0.548578,0.700512,4495,69.297433,70.320227
33389,1.0,1,0.693230,0.695180,2570,80.845829,74.579835
60458,1.0,0,0.542103,0.701212,1795,66.188957,70.474858
53048,1.0,2,0.510558,0.704414,3400,67.339708,69.829174
592,1.0,3,0.533959,0.703955,5695,70.878405,69.463040


id
57094     2
33389     2
60458     1
53048     1
592       1
         ..
25582     1
50013     2
111475    1
71184     2
117473    2
Name: target, Length: 34546, dtype: int64

In [124]:
standart_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()
X[X.columns] = standart_scaler.fit_transform(X[X.columns])
X.head()

,bathrooms,bedrooms,description,features,price,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,
57094,-0.42559,1.306672,0.275262,0.269348,0.023043,-0.001803,0.119173
33389,-0.42559,-0.484290,1.451747,0.238404,-0.050133,0.890837,0.627071
60458,-0.42559,-1.379770,0.222599,0.273414,-0.079594,-0.242074,0.137611
53048,-0.42559,0.411191,-0.033960,0.292001,-0.018582,-0.153126,0.060622
592,-0.42559,1.306672,0.156364,0.289333,0.068660,0.120399,0.016966


In [113]:
X[:20]

,bathrooms,bedrooms,description,features,price,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,
57094,-0.425590,1.306672,0.275262,0.269348,0.023043,-0.001803,0.119173
33389,-0.425590,-0.484290,1.451747,0.238404,-0.050133,0.890837,0.627071
60458,-0.425590,-1.379770,0.222599,0.273414,-0.079594,-0.242074,0.137611
53048,-0.425590,0.411191,-0.033960,0.292001,-0.018582,-0.153126,0.060622
592,-0.425590,1.306672,0.156364,0.289333,0.068660,0.120399,0.016966
31731,-0.425590,-0.484290,0.217041,0.280671,-0.050894,0.009867,-0.271816
12930,-0.425590,0.411191,0.327376,0.253275,-0.015161,-0.001803,0.017759
25585,-0.425590,-0.484290,-0.265673,0.238616,-0.098411,-5.280891,-3.009050
103527,-0.425590,-0.484290,-0.420708,0.274447,-0.075603,0.890837,0.474533


In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)

In [127]:
log_reg = LogisticRegression() 
log_reg.fit(X_train, y_train)

LogisticRegression()

In [128]:
y_pred = log_reg.predict(X_test)
y_pred

array([1, 1, 1, ..., 3, 1, 1])

In [130]:
accuracy_score(y_test, y_pred)

0.7583944423002702

In [132]:
y_dumb = np.ones(len(y_pred))
accuracy_score(y_test, y_dumb)

0.6794673871092243